In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hereutil import here, add_to_sys_path
add_to_sys_path(here())

In [7]:
from src.common_basis import *
import pandas as pd

In [6]:
eng, con = get_connection()

In [99]:
from sqlalchemy import text

df = pd.read_sql(text("""
SELECT record_number,value
FROM vd17.vd17_a
WHERE field_code="034D" AND
record_number NOT IN (
  SELECT record_number FROM vd17.vd17_a
  WHERE field_code="034D"
  GROUP BY record_number
  HAVING COUNT(*)>1
)
"""), con)
df

,record_number,value
0,1,Umfangsangabe
1,3,304 S.
2,4,"[2] Bl., 192 S"
3,5,176 S
4,6,[2] Bl.
...,...,...
305361,310817,[81] Bl
305362,310818,[2] Bl
305363,310819,[1] Bl
305364,310820,[1] Bl


In [102]:
import re
import roman
pageinfo = []
def parse(value: str) -> int | None:
    pages = 0
    v = value.upper()
    v = re.sub("[1-9][0-9]*( S[^,]+?)?, DAS HEISST", "", v)
    v = re.sub("[1-9][0-9]* *\\[[^\\]]*?([1-9][0-9]*)\\]","\\1", v)
    v = re.sub("\\[([1-9][0-9]*\\??)\\]", "\\1", v)
    v = re.sub("\\b(M{1,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|C?D|D?C{1,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|X?L|L?X{1,3})(IX|IV|V?I{0,3})|M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|I?V|V?I{1,3}))\\b",lambda rn: str(roman.fromRoman(rn.group(1))), v)
    while True:
        m = re.search("(S|B)[^ ]* *([1-9][0-9]*) ?- ?([1-9][0-9]*)", v)
        if m is None:
            break
        if m.group(1)=="S":
            multiplier = 1
        elif m.group(1)=="B":
            multiplier = 2
        else:
            raise ValueError()
        pages += multiplier*(int(m.group(3))-int(m.group(2)))
        v = v[:m.start()] + v[m.end()+1:]
    while True:
        m = re.search("((?:[1-9][0-9]* *, *)*)([1-9][0-9]*)[^,0-9]*?(S|B)[^,]*", v)
        if m is None:
            break
        if m.group(3)=="S":
            multiplier = 1
        elif m.group(3)=="B":
            multiplier = 2
        else:
            raise ValueError()
        if m.group(1) != "":
            v2 = m.group(1)
            while True:
                m2 = re.search("([1-9][0-9]*) *, *", v2)
                if m2 is None:
                    break
                pages += int(m2.group(1))*multiplier
                v2 = v2[:m2.start()] + v2[m2.end()+1:]
        pages += int(m.group(2))*multiplier
        v = v[:m.start()] + v[m.end()+1:]
    if v.strip()!="" or pages==0:
        return None
    return pages

pages = []
for record_number, value in df.itertuples(index=False):
    page_count = parse(value)
    if page_count is not None:
        pages.append((record_number,page_count))
tdf = pd.DataFrame(pages, columns=["record_number","page_count"], dtype=pd.Int32Dtype())
tdf

,record_number,page_count
0,3,304
1,4,196
2,5,176
3,6,4
4,7,2
...,...,...
304057,310817,162
304058,310818,4
304059,310819,2
304060,310820,2


In [103]:
set_session_storage_engine(con, "Aria")
tdf.to_sql("vd17_page_counts_a", con, index=False, if_exists="replace")
con.execute(text("ALTER TABLE vd17_page_counts_a ADD UNIQUE INDEX (record_number)"))
con.execute(text("CREATE TABLE vd17_page_counts_c AS SELECT * FROM vd17_page_counts_a WHERE 0=1"))
con.execute(text("INSERT INTO vd17_page_counts_c SELECT * FROM vd17_page_counts_a"))